# Observe other vision practices that have moved to large messaging in Spine

On the 23rd March 2021, three more Vision practices enabled large messaging for GP2GP. We want to look at our Spine data for these practices to identify how their failure rates have changed since moving to large messaging. 

### Scope

Analyse spine data for the following three practices, for transfers both in and out:

Went live with large messaging from: 23.03.21 - 12.16pm 

User ID | Practice Code | Practice Name

#668 | M89019 | Hobs Moat Medical Centre

#33018 | M82010 | Drayton Medical Centre

#857 | H84023 | Essex House Surgery

- Document the technical success rate of their transfers for three months prior to 23.03
- Document a high level breakdown of reasons for failures for this time period.
- Document the technical success rate of their transfers since 23.03. 
- Document a high level breakdown of reasons for this time period

 
### Acceptance Criteria
- We are able to compare the technical success rate for these two practices before and after enabling attachments, and communicate to stakeholders how much their failure rates have changed. 
- We are able to tell the Vision team the reasons for some transfers still failing since moving to Large messaging

In [14]:
from datetime import datetime
practice_codes=["M89019","M82010","H84023"]
time_of_switch=datetime(year=2021, month=3, day=23, hour=12, minute=16)

In [6]:
import pandas as pd
import numpy as np

transfer_file_location = "s3://prm-gp2gp-data-sandbox-dev/transfers-duplicates-hypothesis/"
transfer_files = [
    "9-2020-transfers.parquet",
    "10-2020-transfers.parquet",
    "11-2020-transfers.parquet",
    "12-2020-transfers.parquet",
    "1-2021-transfers.parquet",
    "2-2021-transfers.parquet"
]
transfer_input_files = [transfer_file_location + f for f in transfer_files]
transfers_raw = pd.concat((
    pd.read_parquet(f)
    for f in transfer_input_files
))
# This is only needed when using transfers-duplicates-hypothesis datasets
transfers_raw = transfers_raw.drop(["sending_supplier", "requesting_supplier"], axis=1)




# Given the findings in PRMT-1742 - many duplicate EHR errors are misclassified, the below reclassifies the relevant data
successful_transfers_bool = transfers_raw['request_completed_ack_codes'].apply(lambda x: True in [(np.isnan(i) or i==15) for i in x])
transfers = transfers_raw.copy()
transfers.loc[successful_transfers_bool, "status"] = "INTEGRATED"

# Pending (Sender Error) Code
pending_sender_error_codes=[6,7,10,24,30,23,14,99]
transfers_with_pending_sender_code_bool=transfers['sender_error_code'].isin(pending_sender_error_codes)
transfers_with_pending_with_error_bool=transfers['status']=='PENDING_WITH_ERROR'
transfers_which_need_pending_to_failure_change_bool=transfers_with_pending_sender_code_bool & transfers_with_pending_with_error_bool
transfers.loc[transfers_which_need_pending_to_failure_change_bool,'status']='FAILED'

# Add integrated Late status
eight_days_in_seconds=8*24*60*60
transfers_after_sla_bool=transfers['sla_duration']>eight_days_in_seconds
transfers_with_integrated_bool=transfers['status']=='INTEGRATED'
transfers_integrated_late_bool=transfers_after_sla_bool & transfers_with_integrated_bool
transfers.loc[transfers_integrated_late_bool,'status']='INTEGRATED LATE'


# Supplier name mapping
supplier_renaming = {
    "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)":"EMIS",
    "IN PRACTICE SYSTEMS LTD":"Vision",
    "MICROTEST LTD":"Microtest",
    "THE PHOENIX PARTNERSHIP":"TPP",
    None: "Unknown"
}

asid_lookup_file = "s3://prm-gp2gp-data-sandbox-dev/asid-lookup/asidLookup-Mar-2021.csv.gz"
asid_lookup = pd.read_csv(asid_lookup_file)
lookup = asid_lookup[["ASID", "MName", "NACS","OrgName"]]

transfers = transfers.merge(lookup, left_on='requesting_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'MName': 'requesting_supplier', 'ASID': 'requesting_supplier_asid', 'NACS': 'requesting_ods_code'}, axis=1)
transfers = transfers.merge(lookup, left_on='sending_practice_asid',right_on='ASID',how='left')
transfers = transfers.rename({'MName': 'sending_supplier', 'ASID': 'sending_supplier_asid', 'NACS': 'sending_ods_code'}, axis=1)

transfers["sending_supplier"] = transfers["sending_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["requesting_supplier"] = transfers["requesting_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())

### Extract the data relevant to the practices in question

In [12]:
requesting_transfers_bool=transfers["requesting_ods_code"].isin(practice_codes)
requesting_transfers=transfers.loc[requesting_transfers_bool].copy()
requesting_transfers['ods_code']=requesting_transfers['requesting_ods_code']
requesting_transfers['transfer type']='Requesting'

sending_transfers_bool=transfers["sending_ods_code"].isin(practice_codes)
sending_transfers=transfers.loc[sending_transfers_bool].copy()
sending_transfers['ods_code']=sending_transfers['sending_ods_code']
sending_transfers['transfer type']='Sending'

relevant_transfers=pd.concat([requesting_transfers,sending_transfers],axis=0)

In [18]:
relevant_transfers['Large Messaging']=relevant_transfers['date_requested']>time_of_switch


2187